# LLM: Zero-shot classification through LLMs and prompts

Classify data into categories without being explicitly trained on labeled examples for that specific task. Instead, the model uses pre-trained knowledge and natural language prompts to infer the correct category.

| Prompt Name              | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|--------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **simple_prompt**        | "Does the following person develop a psychological disorder between time point T1 and T2? Data: …"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| **class_definitions_prompt** | "Given the following data, classify whether this person develops a psychological disorder between T1 and T2 according to the instructions provided and data measured by F-DIPS structural interviews. Respond with YES or NO. Instructions: NO: The person did not develop any new psychological disorder between T1 and T2. This means they were either healthy at both time points, had an ongoing disorder across both time points, or had already recovered from a previous disorder. YES: The person was psychologically healthy at T1 but developed a psychological disorder at T2. Data: …"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| **profiled_simple_prompt** | "You are an expert in clinical psychology and mental health diagnostics. You are trained to analyze longitudinal data to assess whether a person develops a psychological disorder over time. You know how to analyze data measured with F-DIPS structural interviews at two time points T1 and T2. Does the following person develop a psychological disorder between time point T1 and T2? Data: …"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| **few_shot_prompt**      | "Please refer to the following examples of correctly classified data points with 'Total mental disorders incidence point prevalence' being the true classification: … Based on the previous example data prompts, classify the following data. Does the following person develop a psychological disorder between time point T1 and T2? Data: …"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| **vignette_text_prompt** | "A woman with a BMI of {bmi}, an education level of {education}, and a socioeconomic status of {ses} has the following psychological profile: At Time Point 1, she showed {t1_pm_health} positive mental health, {t1_support} social support, and {t1_self_eff} general self-efficacy. Her life satisfaction was {t1_life_sat}, and she relied on {t1_coping_prob} problem-focused and {t1_coping_emot} emotion-focused coping strategies. Anxiety sensitivity was {t1_anx_sens}, and her fear of bodily sensations was {t1_fear}. She reported {t1_dysfunc} levels of dysfunctional attitudes and {t1_gsi} levels of general psychopathology. Her stress level was {t1_stress}. By Time Point 2, approximately 17 months later, she reported {t2_pm_health} positive mental health, {t2_support} social support, and {t2_self_eff} self-efficacy. Life satisfaction remained {t2_life_sat}. However, anxiety sensitivity had become {t2_anx_sens}, and fear of bodily sensations was {t2_fear}. Dysfunctional attitudes were {t2_dysfunc}, and general psychopathology was {t2_gsi}. Does this person develop a psychological disorder between time point T1 and T2?" |

**Models**:

- GPT-4o (OpenAI)
- Gemini (Google)
- Gemma (Google)
- Llama (Meta)
- Claude (Anthropic)
- DeepSeek


## 0 Imports

In [43]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from google import genai
from google.genai import types
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import pdist

In [42]:
# data = pd.read_csv("../dat/dips/DIPS_Data_cleaned.csv", sep = ",", low_memory = False)
data_change = pd.read_csv("../dat/dips/DIPS_Data_cleaned_change.csv", sep = ",", low_memory = False)
# data_pred = pd.read_csv("../dat/dips/DIPS_Data_cleaned_pred.csv", sep = ",", low_memory = False)
# data_pred_y = pd.read_csv("../dat/dips/DIPS_Data_cleaned_pred_y.csv", sep = ",", low_memory = False)

## 1 Zero-shot prompts

In this section, I will **create different prompts for zero-shot classification** for every person (row) in the test set. The prompts will be used to classify whether a person develops a psychological disorder between time point T1 and T2.

### 1.1 Data preparation

In [3]:
# Predictors
X = data_change
X = X.drop(["hpi"], axis = 1)

# Target
y = data_change["hpi"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# concatenate X and y for example data
dat_train = pd.concat([X_train, y_train], axis = 1)

print("LLMs \n",
      "X_train shape: ", X_train.shape, round(X_train.shape[0]/len(X), 2), "\n",
      "X_test shape: ", X_test.shape, round(X_test.shape[0]/len(X), 2),  "\n",
      "y_train shape: ", y_train.shape, round(y_train.shape[0]/len(y), 2), "\n",
      "y_test shape: ", y_test.shape, round(y_test.shape[0]/len(y), 2), "\n")

LLMs 
 X_train shape:  (983, 22) 0.8 
 X_test shape:  (246, 22) 0.2 
 y_train shape:  (983,) 0.8 
 y_test shape:  (246,) 0.2 



In [4]:
# get clear names for each column in data
preds = X.columns
preds

Index(['whi_ges', 'soz_ges', 'gke_ges', 'lzh_ges', 'ile', 'woc_gesp',
       'woc_gese', 'asi_ges', 'bsq_ges', 'das_ges', 'scl_gsi', 'bild',
       'bmi_kat', 'ses_kom', 'whi_change', 'soz_change', 'gke_change',
       'lzh_change', 'asi_change', 'bsq_change', 'das_change', 'scl_change'],
      dtype='object')

In [5]:
# predictor_names = ["T1 Positive mental health",    #whi_ges
#                    "T1 Social support",            #soz_ges (SAVE)
#                    "T1 General self-efficacy",     #gke_ges SAVE
#                    "T1 Life satisfaction",         #lzh_ges SAVE
#                    "T1 Problem-focused coping",    #woc_gesp SAVE
#                    "T1 Emotion-focused coping",    #woc_gese SAVE
#                    "T1 Anxiety sensitivity",       #asi_ges SAVE
#                    "T1 Fear of bodily sensations", #bsq_ges SAVE
#                    "T1 Dysfunctional attitudes",   #das_ges SAVE
#                    "T1 General psychopathology: Global Severity Index (GSI)",    #scl_gsi SAVE
#                    "Education",                    #bild
#                    "T1 BMI",                       #bmi_kat
#                    "Socioeconomic status",         #ses_kom
#                    "T2 Positive mental health",    #whi_gesy
#                    "T2 Social support",            #soz_gesy (SAVE)
#                    "T2 General self-efficacy",     #gke_gesy SAVE
#                    "T2 Life satisfaction",         #lzh_gesy SAVE
#                    "T2 Anxiety sensitivity",       #asi_gesy SAVE
#                    "T2 Fear of bodily sensations", #bsq_gesy SAVE
#                    "T2 Dysfunctional attitudes",   #das_gesy SAVE
#                    "T2 General psychopathology: Global Severity Index (GSI)",    #scl_gsi SAVE
#                    "T1 Stress"                     #ile SAVE
#                    ]

predictor_names = ["T1 Positive mental health",    #whi_ges
                   "T1 Social support",            #soz_ges (SAVE)
                   "T1 General self-efficacy",     #gke_ges SAVE
                   "T1 Life satisfaction",         #lzh_ges SAVE
                   "T1 Stress",                     #ile SAVE
                   "T1 Problem-focused coping",    #woc_gesp SAVE
                   "T1 Emotion-focused coping",    #woc_gese SAVE
                   "T1 Anxiety sensitivity",       #asi_ges SAVE
                   "T1 Fear of bodily sensations", #bsq_ges SAVE
                   "T1 Dysfunctional attitudes",   #das_ges SAVE
                   "T1 General psychopathology: Global Severity Index (GSI)",    #scl_gsi SAVE
                   "Education",                    #bild
                   "T1 BMI",                       #bmi_kat
                   "Socioeconomic status",         #ses_kom
                   "Change in Positive mental health (T2-T1)",    #whi_change
                   "Change in Social support (T2-T1)",            #soz_change
                   "Change in General self-efficacy (T2-T1)",     #gke_change
                   "Change in Life satisfaction (T2-T1)",         #lzh_change
                   "Change in Anxiety sensitivity (T2-T1)",       #asi_change
                   "Change in Fear of bodily sensations (T2-T1)", #bsq_change
                   "Change in Dysfunctional attitudes (T2-T1)",   #das_change
                   "Change in General psychopathology: Global Severity Index (GSI) (T2-T1)",    #scl_change
                   ]

target_names = ["Total mental disorders incidence point prevalence"]    #hpi

In [6]:
# print col and names
for col, name in zip(preds, predictor_names):
    print(f"{col}: {name}")

whi_ges: T1 Positive mental health
soz_ges: T1 Social support
gke_ges: T1 General self-efficacy
lzh_ges: T1 Life satisfaction
ile: T1 Stress
woc_gesp: T1 Problem-focused coping
woc_gese: T1 Emotion-focused coping
asi_ges: T1 Anxiety sensitivity
bsq_ges: T1 Fear of bodily sensations
das_ges: T1 Dysfunctional attitudes
scl_gsi: T1 General psychopathology: Global Severity Index (GSI)
bild: Education
bmi_kat: T1 BMI
ses_kom: Socioeconomic status
whi_change: Change in Positive mental health (T2-T1)
soz_change: Change in Social support (T2-T1)
gke_change: Change in General self-efficacy (T2-T1)
lzh_change: Change in Life satisfaction (T2-T1)
asi_change: Change in Anxiety sensitivity (T2-T1)
bsq_change: Change in Fear of bodily sensations (T2-T1)
das_change: Change in Dysfunctional attitudes (T2-T1)
scl_change: Change in General psychopathology: Global Severity Index (GSI) (T2-T1)


### 1.2 Simple prompt

In [7]:
def create_simple_prompt(data_row):
    """
    Create a simple prompt for zero-shot classification.
    """

    # Convert the data row to a string format using the real column names
    data_string = ", ".join([f"{name}: {value}" for name, value in zip(predictor_names, data_row)])

    instruction = "Respond only with YES or NO."

    prompt = f"Does the following person develop a psychological disorder between time point T1 and T2? Data: {data_string}"

    return instruction, prompt

In [8]:
# # Test the function
# data_row = X.iloc[4]
# simple_instrcution, simple_prompt = create_simple_prompt(data_row)
# print(simple_prompt)
# print("\n", target_names[0], y.iloc[4])

In [9]:
# Create a simple prompt for each row in the test set
X_test_simple_prompt = []

for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    simple_instrcution, simple_prompt = create_simple_prompt(data_row)
    X_test_simple_prompt.append(simple_prompt)

X_test_simple_prompt[0]

'Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.9943983599434336, T1 Social support: 0.9009877265364608, T1 General self-efficacy: 0.3649793457412237, T1 Life satisfaction: 1.1905292237027845, T1 Stress: -1.5578412804282529, T1 Problem-focused coping: -1.6948315955326445, T1 Emotion-focused coping: -1.2508043631414372, T1 Anxiety sensitivity: -0.3690317323674431, T1 Fear of bodily sensations: 0.9550793433276124, T1 Dysfunctional attitudes: -1.4754544059868735, T1 General psychopathology: Global Severity Index (GSI): -1.0723188363175855, Education: 2.0, T1 BMI: 2.0, Socioeconomic status: 2.0, Change in Positive mental health (T2-T1): 0.7978354238065312, Change in Social support (T2-T1): -0.349660653077381, Change in General self-efficacy (T2-T1): 0.3709329069367079, Change in Life satisfaction (T2-T1): 0.2687168967469017, Change in Anxiety sensitivity (T2-T1): -0.7236998319770478, Change in Fear of bodily sens

In [10]:
# # create random input for testing
data_row = X_test.iloc[4]

# add a random number to the data_row
data_row = data_row + np.random.rand(len(data_row))

simple_instruction, simple_prompt = create_simple_prompt(data_row)
print(simple_prompt)

Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.1685026228625055, T1 Social support: 0.1292773534581788, T1 General self-efficacy: 2.2186828447235936, T1 Life satisfaction: 1.2972818244029787, T1 Stress: -0.4477858695575503, T1 Problem-focused coping: 1.2352789569078684, T1 Emotion-focused coping: -0.19254052323905468, T1 Anxiety sensitivity: 1.3265453166093382, T1 Fear of bodily sensations: -0.6049117036038222, T1 Dysfunctional attitudes: 0.18861759909736087, T1 General psychopathology: Global Severity Index (GSI): -0.2050144065883981, Education: 2.294759641475654, T1 BMI: 1.146459682985284, Socioeconomic status: 3.434049679180034, Change in Positive mental health (T2-T1): 0.5394210479985893, Change in Social support (T2-T1): 1.9915141646651962, Change in General self-efficacy (T2-T1): 1.277769364162891, Change in Life satisfaction (T2-T1): 0.8666641617538559, Change in Anxiety sensitivity (T2-T1): 0.15140779

### 1.3 Class definitions prompt

In [11]:
def create_class_definitions_prompt(data_row):
    """
    Create a class definitions prompt for zero-shot classification.
    """

    # Convert the data row to a string format using the real column names
    data_string = ", ".join([f"{name}: {value}" for name, value in zip(predictor_names, data_row)])

    instruction = "Respond only with YES or NO."

    prompt = f"Given the following data, classify whether this person develops a psychological disorder between T1 and T2 according to the instructions provided and data measured by F-DIPS structural interviews. Respond with YES or NO. Instructions: NO: The person did not develop any new psychological disorder between T1 and T2. This means they were either healthy at both time points, had an ongoing disorder across both time points, or had already recovered from a previous disorder. YES: The person was psychologically healthy at T1 but developed a psychological disorder at T2. Does the following person develop a psychological disorder between time point T1 and T2? Data: {data_string}"

    return instruction, prompt

In [12]:
# # Test the function
# data_row = X.iloc[4]
# class_definitions_instruction, class_definitions_prompt = create_class_definitions_prompt(data_row)
# print(class_definitions_prompt)
# print("\n", target_names[0], y.iloc[4])

In [13]:
# Create a class definitions prompt for each row in the test set
X_test_class_definitions_prompt = []
for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    class_definitions_instruction, class_definitions_prompt = create_class_definitions_prompt(data_row)
    X_test_class_definitions_prompt.append(class_definitions_prompt)

X_test_class_definitions_prompt[0]

'Given the following data, classify whether this person develops a psychological disorder between T1 and T2 according to the instructions provided and data measured by F-DIPS structural interviews. Respond with YES or NO. Instructions: NO: The person did not develop any new psychological disorder between T1 and T2. This means they were either healthy at both time points, had an ongoing disorder across both time points, or had already recovered from a previous disorder. YES: The person was psychologically healthy at T1 but developed a psychological disorder at T2. Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.9943983599434336, T1 Social support: 0.9009877265364608, T1 General self-efficacy: 0.3649793457412237, T1 Life satisfaction: 1.1905292237027845, T1 Stress: -1.5578412804282529, T1 Problem-focused coping: -1.6948315955326445, T1 Emotion-focused coping: -1.2508043631414372, T1 Anxiety sensitivity: -0.36903

In [14]:
# # create random input for testing
# data_row = np.random.rand(len(predictor_names))
# class_definitions_instruction, class_definitions_prompt = create_class_definitions_prompt(data_row)
# print(class_definitions_prompt)

### 1.4 Profiled simple prompt

In [15]:
def create_profiled_simple_prompt(data_row):
    """
    Create a profiled simple prompt for zero-shot classification.
    """

    # Convert the data row to a string format using the real column names
    data_string = ", ".join([f"{name}: {value}" for name, value in zip(predictor_names, data_row)])

    instruction = "Respond only with YES or NO."

    prompt = f"You are an expert in clinical psychology and mental health diagnostics. You are trained to analyze longitudinal data to assess whether a person develops a psychological disorder over time. You know how to analyze data measured with F-DIPS structural interviews at two time points T1 and T2. Does the following person develop a psychological disorder between time point T1 and T2? Data: {data_string}"

    return instruction, prompt

In [16]:
# # Test the function
# data_row = X.iloc[4]
# profiled_simple_instruction, profiled_simple_prompt = create_profiled_simple_prompt(data_row)
# print(profiled_simple_prompt)
# print("\n", target_names[0], y.iloc[4])

In [17]:
# Create a profiled simple prompt for each row in the test set
X_test_profiled_simple_prompt = []
for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    profiled_simple_instruction, profiled_simple_prompt = create_profiled_simple_prompt(data_row)
    X_test_profiled_simple_prompt.append(profiled_simple_prompt)

X_test_profiled_simple_prompt[0]

'You are an expert in clinical psychology and mental health diagnostics. You are trained to analyze longitudinal data to assess whether a person develops a psychological disorder over time. You know how to analyze data measured with F-DIPS structural interviews at two time points T1 and T2. Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.9943983599434336, T1 Social support: 0.9009877265364608, T1 General self-efficacy: 0.3649793457412237, T1 Life satisfaction: 1.1905292237027845, T1 Stress: -1.5578412804282529, T1 Problem-focused coping: -1.6948315955326445, T1 Emotion-focused coping: -1.2508043631414372, T1 Anxiety sensitivity: -0.3690317323674431, T1 Fear of bodily sensations: 0.9550793433276124, T1 Dysfunctional attitudes: -1.4754544059868735, T1 General psychopathology: Global Severity Index (GSI): -1.0723188363175855, Education: 2.0, T1 BMI: 2.0, Socioeconomic status: 2.0, Change in Positive mental health

In [18]:
# # create random input for testing
# data_row = np.random.rand(len(predictor_names))
# profiled_simple_instruction, profiled_simple_prompt = create_profiled_simple_prompt(data_row)
# print(profiled_simple_prompt)

### 1.5 Few-shot prompt

In [19]:
def create_few_shot_prompt(data_row, few_shot_examples):
    """
    Create a few-shot prompt for zero-shot classification.
    """

    example_data = []

    # create example prompt but including the target aswell, stating Example 1: ..., Example 2: ...
    for i, example in enumerate(few_shot_examples):
        example_data.append(
            f"Example {i + 1}: " + ", ".join([f"{name}: {value}" for name, value in zip(predictor_names + target_names, example)])
        )

    # Convert the data row to a string format using the real column names
    data_string = ", ".join([f"{name}: {value}" for name, value in zip(predictor_names, data_row)])

    instruction = "Respond only with YES or NO."

    prompt1 = f"Please refer to the following examples of correctly classified data points with 'Total mental disorders incidence point prevalence' being the true classification: {example_data}"

    prompt2 = f"Based on the previous example data prompts, classify the following data. Does the following person develop a psychological disorder between time point T1 and T2? Data: {data_string}"

    # concatenate
    prompt = f"{prompt1} {prompt2}"

    return instruction, prompt

In [20]:
def get_similar_examples(data_row, dat_train, n):
    """
    Get the top-n semantically similar examples to the test product from the training set.
    """

    # get columns that we want to use for similarity
    preds = dat_train.drop("hpi", axis = 1).columns
    dat_preds = dat_train[preds]
    data_row = data_row[preds]
    data_row = np.array([data_row])

    # dictionary to store the distances
    distances = {}

    for row in dat_preds.iterrows():
        # create array with the row and the data_row
        row_input = np.array([row[1]])
        dist_row = np.concatenate((data_row, row_input), axis = 0)

        # calculate the distance between the two rows
        dist = pdist(dist_row, metric = "euclidean")

        # store the distance in the dictionary
        distances[row[0]] = dist[0]

    # sort the dictionary by distance
    sorted_distances = sorted(distances.items(), key = lambda x: x[1])

    # get top keys
    keys = []
    for key in sorted_distances[:n]:
        keys.append(key[0])

    # drop all rows that are not in keys
    ex_dat = dat_train.loc[keys]

    return ex_dat

In [21]:
# # Test the function
# close_ex = get_similar_examples(X.iloc[4], dat_train, 5)
#
# data_row = X.iloc[4]
# few_shot_examples = close_ex.values
# few_shot_instruction, few_shot_prompt = create_few_shot_prompt(data_row, few_shot_examples)
# print(few_shot_prompt)

In [22]:
# Create a few-shot prompt for each row in the test set
X_test_few_shot_prompt = []
for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    few_shot_examples = get_similar_examples(data_row, dat_train, 5).values
    few_shot_instruction, few_shot_prompt = create_few_shot_prompt(data_row, few_shot_examples)
    X_test_few_shot_prompt.append(few_shot_prompt)

X_test_few_shot_prompt[0]

"Please refer to the following examples of correctly classified data points with 'Total mental disorders incidence point prevalence' being the true classification: ['Example 1: T1 Positive mental health: -0.9943983599434336, T1 Social support: 0.6339015451407742, T1 General self-efficacy: 0.3649793457412237, T1 Life satisfaction: 0.1754672017922696, T1 Stress: -1.3578635351131991, T1 Problem-focused coping: -0.2011120086996256, T1 Emotion-focused coping: -1.331839606278012, T1 Anxiety sensitivity: 0.5557512543954795, T1 Fear of bodily sensations: 1.5282544252035275, T1 Dysfunctional attitudes: -0.7669093694574932, T1 General psychopathology: Global Severity Index (GSI): -0.8907285788068687, Education: 1.0, T1 BMI: 2.0, Socioeconomic status: 2.0, Change in Positive mental health (T2-T1): 0.7978354238065312, Change in Social support (T2-T1): -0.460474567131066, Change in General self-efficacy (T2-T1): -0.1819798191096402, Change in Life satisfaction (T2-T1): 0.6686513409272705, Change in

In [23]:
# # create random input for testing
# data_row = X_test.iloc[4]
# # add a random number to the data_row
# data_row = data_row + np.random.rand(len(data_row))
#
# few_shot_examples = get_similar_examples(data_row, dat_train, 5).values
# few_shot_instruction, few_shot_prompt = create_few_shot_prompt(data_row, few_shot_examples)
# print(few_shot_prompt)

### 1.6 Vignette text prompt

In [24]:
def interpret_value(val, thresh_low = -1, thresh_high = 1):
    """Convert standardized value to qualitative description with value."""
    val_str = f"{val:.3f}"
    if val > thresh_high:
        return f"above average ({val_str})"
    elif val < thresh_low:
        return f"below average ({val_str})"
    else:
        return f"average ({val_str})"

In [25]:
def interpret_change(val, thresh_low = -1, thresh_high = 1):
    """Convert standardized value to qualitative description with value."""
    val_str = f"{val:.3f}"
    if val > thresh_high:
        return f"increased ({val_str})"
    elif val < thresh_low:
        return f"decreased ({val_str})"
    else:
        return f"similar ({val_str})"

In [26]:
def create_vignette_prompt(data):

    # demographics
    bmi = data['bmi_kat']
    education = data['bild']
    ses = data['ses_kom']

    # time point 1
    t1_pm_health = interpret_value(data['whi_ges'])
    t1_support = interpret_value(data['soz_ges'])
    t1_self_eff = interpret_value(data['gke_ges'])
    t1_life_sat = interpret_value(data['lzh_ges'])
    t1_coping_prob = interpret_value(data['woc_gesp'])
    t1_coping_emot = interpret_value(data['woc_gese'])
    t1_anx_sens = interpret_value(data['asi_ges'])
    t1_fear = interpret_value(data['bsq_ges'])
    t1_dysfunc = interpret_value(data['das_ges'])
    t1_gsi = interpret_value(data['scl_gsi'])
    t1_stress = interpret_value(data['ile'])

    # # time point 2
    # t2_pm_health = interpret_value(data['whi_gesy'])
    # t2_support = interpret_value(data['soz_gesy'])
    # t2_self_eff = interpret_value(data['gke_gesy'])
    # t2_life_sat = interpret_value(data['lzh_gesy'])
    # t2_anx_sens = interpret_value(data['asi_gesy'])
    # t2_fear = interpret_value(data['bsq_gesy'])
    # t2_dysfunc = interpret_value(data['das_gesy'])
    # t2_gsi = interpret_value(data['scl_gsiy'])

    # change variables
    change_pm_health = interpret_change(data['whi_change'])
    change_support = interpret_change(data['soz_change'])
    change_self_eff = interpret_change(data['gke_change'])
    change_life_sat = interpret_change(data['lzh_change'])
    change_anx_sens = interpret_change(data['asi_change'])
    change_fear = interpret_change(data['bsq_change'])
    change_dysfunc = interpret_change(data['das_change'])
    change_gsi = interpret_change(data['scl_change'])

    instruction = "Respond only with YES or NO."

    # generate prompt
    # vignette = f"""A woman with a BMI of {bmi}, an education level of {education}, and a socioeconomic status of {ses} has the following psychological profile: At Time Point 1, she showed {t1_pm_health} positive mental health, {t1_support} social support, and {t1_self_eff} general self-efficacy. Her life satisfaction was {t1_life_sat}, and she relied on {t1_coping_prob} problem-focused and {t1_coping_emot} emotion-focused coping strategies. Anxiety sensitivity was {t1_anx_sens}, and her fear of bodily sensations was {t1_fear}. She reported {t1_dysfunc} levels of dysfunctional attitudes and {t1_gsi} levels of general psychopathology. Her stress level was {t1_stress}. By Time Point 2, approximately 17 months later, she reported {t2_pm_health} positive mental health, {t2_support} social support, and {t2_self_eff} self-efficacy. Life satisfaction remained {t2_life_sat}. However, anxiety sensitivity had become {t2_anx_sens}, and fear of bodily sensations was {t2_fear}. Dysfunctional attitudes were {t2_dysfunc}, and general psychopathology was {t2_gsi}. Does this person develop a psychological disorder between time point T1 and T2?"""

    vignette = f"""A woman with a BMI of {bmi}, an education level of {education}, and a socioeconomic status of {ses} has the following psychological profile: At time point 1, she showed {t1_pm_health} positive mental health, {t1_support} social support, and {t1_self_eff} general self-efficacy. Her life satisfaction was {t1_life_sat}, and she relied on {t1_coping_prob} problem-focused and {t1_coping_emot} emotion-focused coping strategies. Anxiety sensitivity was {t1_anx_sens}, and her fear of bodily sensations was {t1_fear}. She reported {t1_dysfunc} levels of dysfunctional attitudes and {t1_gsi} levels of general psychopathology. Her stress level was {t1_stress}. By time point 2, approximately 17 months later, she reported {change_pm_health} positive mental health, {change_support} social support, and {change_self_eff} self-efficacy. Life satisfaction was {change_life_sat}. Anxiety sensitivity was reported to be {change_anx_sens}, and fear of bodily sensations was {change_fear}. Dysfunctional attitudes were {change_dysfunc}, and general psychopathology was {change_gsi} compared to time point 1. Does this person develop a psychological disorder between time point T1 and T2?"""

    return instruction, vignette

In [27]:
# # Test the function
# data_row = X.iloc[4]
# vignette_instruction, vignette_prompt = create_vignette_prompt(data_row)
# print(vignette_prompt)

In [28]:
# Create a vignette prompt for each row in the test set
X_test_vignette_prompt = []
for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    vignette_instruction, vignette_prompt = create_vignette_prompt(data_row)
    X_test_vignette_prompt.append(vignette_prompt)

X_test_vignette_prompt[0]

'A woman with a BMI of 2.0, an education level of 2.0, and a socioeconomic status of 2.0 has the following psychological profile: At time point 1, she showed average (-0.994) positive mental health, average (0.901) social support, and average (0.365) general self-efficacy. Her life satisfaction was above average (1.191), and she relied on below average (-1.695) problem-focused and below average (-1.251) emotion-focused coping strategies. Anxiety sensitivity was average (-0.369), and her fear of bodily sensations was average (0.955). She reported below average (-1.475) levels of dysfunctional attitudes and below average (-1.072) levels of general psychopathology. Her stress level was below average (-1.558). By time point 2, approximately 17 months later, she reported similar (0.798) positive mental health, similar (-0.350) social support, and similar (0.371) self-efficacy. Life satisfaction was similar (0.269). Anxiety sensitivity was reported to be similar (-0.724), and fear of bodily 

In [29]:
# # create random input for testing
# data_row = X_test.iloc[4]
# # add a random number to the data_row
# data_row = data_row + np.random.rand(len(data_row))
#
# vignette_instruction, vignette_prompt = create_vignette_prompt(data_row)
# print(vignette_prompt)

## 2 Zero-shot classification with LLMs

In this section, I will use the prompts created in the previous section to **classify the test set using different LLMs**. The LLMs will be used to classify whether a person develops a psychological disorder between time point T1 and T2.

### 2.1 ChatGPT-4o (OpenAI)

#### 2.1.1 Testing prompting

In [30]:
# client = OpenAI(
#     api_key = os.environ.get("OPENAI_API_KEY"),
# )
#
# # testing
# response = client.responses.create(
#     model = "gpt-4o-mini",
#     instructions = "You are a coding assistant that talks like a pirate.",
#     input = "How do I check if a Python object is an instance of a class?",
# )
#
# print(response.output_text)

In [31]:
X_test_simple_prompt[0]

'Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.9943983599434336, T1 Social support: 0.9009877265364608, T1 General self-efficacy: 0.3649793457412237, T1 Life satisfaction: 1.1905292237027845, T1 Stress: -1.5578412804282529, T1 Problem-focused coping: -1.6948315955326445, T1 Emotion-focused coping: -1.2508043631414372, T1 Anxiety sensitivity: -0.3690317323674431, T1 Fear of bodily sensations: 0.9550793433276124, T1 Dysfunctional attitudes: -1.4754544059868735, T1 General psychopathology: Global Severity Index (GSI): -1.0723188363175855, Education: 2.0, T1 BMI: 2.0, Socioeconomic status: 2.0, Change in Positive mental health (T2-T1): 0.7978354238065312, Change in Social support (T2-T1): -0.349660653077381, Change in General self-efficacy (T2-T1): 0.3709329069367079, Change in Life satisfaction (T2-T1): 0.2687168967469017, Change in Anxiety sensitivity (T2-T1): -0.7236998319770478, Change in Fear of bodily sens

#### 2.1.2. Prompting with ChatGPT-4o

In [32]:
# simple_prompt_array_GPT = []
#
# client = OpenAI(
#     api_key = os.environ.get("OPENAI_API_KEY"),
# )
#
# # iterate over the test set and save the response for each prompt in an array
# for prompt in X_test_simple_prompt:
#     response = client.responses.create(
#         model = "gpt-4o",
#         instructions = simple_instruction,
#         input = prompt,
#     )
#     simple_prompt_array_GPT.append(response.output_text)
#     print(response.output_text)
#
# # value counts for array
# counts_simple_GPT = pd.Series(simple_prompt_array_GPT).value_counts()
# print(counts_simple_GPT)
#
# # convert YES to 1 and NO to 0
# simple_prompt_array_GPT = [1 if response == "YES" else 0 if response == "NO" else np.nan for response in simple_prompt_array_GPT]
# simple_prompt_array_GPT
#
# # save the array to a csv file
# simple_prompt_df_GPT = pd.DataFrame(simple_prompt_array_GPT, columns = ["y_pred"])
# simple_prompt_df_GPT.to_csv("../exp/preds_LLMs/y_pred_GPT4_simple_prompt.csv", sep = ",", index = False)

In [33]:
# class_def_array_GPT = []
#
# client = OpenAI(
#     api_key = os.environ.get("OPENAI_API_KEY"),
# )
#
# # iterate over the test set and save the response for each prompt in an array
# for prompt in X_test_class_definitions_prompt:
#     response = client.responses.create(
#         model = "gpt-4o",
#         instructions = class_definitions_instruction,
#         input = prompt,
#     )
#     class_def_array_GPT.append(response.output_text)
#     print(response.output_text)
#
# # value counts for array
# counts_class_def_GPT = pd.Series(class_def_array_GPT).value_counts()
# print(counts_class_def_GPT)
#
# # convert YES to 1 and NO to 0
# class_def_array_GPT = [1 if response == "YES" else 0 for response in class_def_array_GPT]
# class_def_array_GPT
#
# # save the array to a csv file
# class_def_df_GPT = pd.DataFrame(class_def_array_GPT, columns = ["y_pred"])
# class_def_df_GPT.to_csv("../exp/preds_LLMs/y_pred_GPT4_class_definitions_prompt.csv", sep = ",", index = False)

NO
NO
NO
YES
NO
YES
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
YES
NO
YES
NO
NO
YES
NO
NO
NO
YES
NO
YES
NO
YES
YES
NO
NO
NO
NO
YES
NO
YES
YES
NO
NO
YES
NO
NO
NO
NO
NO
YES
YES
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
YES
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
YES
NO
NO
NO
YES
NO
NO
NO
YES
NO
NO
YES
YES
NO
NO
NO
YES
NO
NO
NO
NO
YES
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
YES
NO
NO
YES
YES
NO
NO
NO
NO
YES
YES
NO
NO
NO
NO
NO
YES
YES
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
YES
YES
NO
YES
NO
NO
YES
NO
NO
YES
NO
YES
NO
NO
NO
YES
YES
NO
YES
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
YES
NO
NO
YES
NO
NO
NO
YES
NO
YES
NO
YES
NO
NO
NO
YES
NO
NO
NO
NO
YES
NO
NO
YES
NO
NO
YES
NO
NO
NO
YES
YES
NO
NO
NO
NO
NO     184
YES     62
Name: count, dtype: int64


In [36]:
# profiled_simple_array_GPT = []
#
# client = OpenAI(
#     api_key = os.environ.get("OPENAI_API_KEY"),
# )
#
# # iterate over the test set and save the response for each prompt in an array
# for prompt in X_test_profiled_simple_prompt:
#     response = client.responses.create(
#         model = "gpt-4o",
#         instructions = profiled_simple_instruction,
#         input = prompt,
#     )
#     profiled_simple_array_GPT.append(response.output_text)
#     print(response.output_text)
#
# # value counts for array
# counts_profiled_simple_GPT = pd.Series(profiled_simple_array_GPT).value_counts()
# print(counts_profiled_simple_GPT)
#
# # convert YES to 1 and NO to 0
# profiled_simple_array_GPT_val = [1 if response == "YES" else 0 for response in profiled_simple_array_GPT]
# profiled_simple_array_GPT_val
#
# # save the array to a csv file
# profiled_simple_df_GPT = pd.DataFrame(profiled_simple_array_GPT_val, columns = ["y_pred"])
# profiled_simple_df_GPT.to_csv("../exp/preds_LLMs/y_pred_GPT4_profiled_simple_prompt.csv", sep = ",", index = False)

NO
NO
NO
YES
NO
NO
YES
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
I'm sorry, I can't determine that.
NO
NO
YES
NO
NO
YES
NO
YES
NO
NO
NO
YES
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
YES
NO
NO
NO
I'm sorry, I can't determine if the person develops a psychological disorder between time point T1 and T2 based on the data provided.
YES
NO
NO
NO
NO
NO
YES
YES
NO
YES
NO
YES
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
YES
YES
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
I'm sorry, I can't determine that.
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
YES
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
I'm sorry, I can't determine that.
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
YES
NO
YES
NO
NO
NO
YES
NO
NO
NO
YES
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
YES
NO
NO
NO
NO
NO

In [37]:
# few_shot_array_GPT = []
#
# client = OpenAI(
#     api_key = os.environ.get("OPENAI_API_KEY"),
# )
#
# # iterate over the test set and save the response for each prompt in an array
# for prompt in X_test_few_shot_prompt:
#     response = client.responses.create(
#         model = "gpt-4o",
#         instructions = few_shot_instruction,
#         input = prompt,
#     )
#     few_shot_array_GPT.append(response.output_text)
#     print(response.output_text)
#
# # value counts for array
# counts_few_shot_GPT = pd.Series(few_shot_array_GPT).value_counts()
# print(counts_few_shot_GPT)
#
# # convert YES to 1 and NO to 0
# few_shot_array_GPT_val = [1 if response == "YES" else 0 for response in few_shot_array_GPT]
# few_shot_array_GPT_val
#
# # save the array to a csv file
# few_shot_df_GPT = pd.DataFrame(few_shot_array_GPT_val, columns = ["y_pred"])
# few_shot_df_GPT.to_csv("../exp/preds_LLMs/y_pred_GPT4_few_shot_prompt.csv", sep = ",", index = False)

NO
NO
NO
YES
NO
NO
NO
NO
NO
YES
YES
YES
YES
NO
NO
NO
NO
YES
NO
NO
YES
YES
NO
NO
NO
NO
NO
YES
YES
YES
NO
YES
NO
NO
YES
NO
NO
NO
NO
YES
YES
NO
YES
NO
NO
YES
NO
NO
NO
NO
YES
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
YES
NO
NO
YES
NO
NO
NO
NO
NO
YES
YES
YES
NO
NO
YES
NO
YES
NO
NO
NO
NO
YES
NO
NO
NO
YES
YES
YES
NO
NO
YES
NO
NO
NO
YES
YES
NO
YES
NO
NO
YES
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
YES
NO
NO
NO
YES
NO
YES
NO
YES
YES
NO
NO
YES
NO
NO
NO
YES
YES
NO
NO
YES
NO
NO
YES
NO
NO
NO
NO
YES
YES
NO
NO
YES
NO
NO
NO
NO
NO
YES
NO
YES
YES
YES
NO
NO
YES
NO
YES
NO
YES
YES
NO
YES
NO
YES
NO
YES
YES
NO
YES
YES
NO
NO
NO
NO
YES
NO
YES
YES
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
YES
NO
YES
NO
NO
YES
YES
YES
NO
YES
NO
YES
YES
NO
YES
NO
YES
YES
NO
YES
NO
NO
YES
NO
NO
YES
NO
NO     160
YES     86
Name: count, dtype: int64


In [38]:
# vignette_array_GPT = []
#
# client = OpenAI(
#     api_key = os.environ.get("OPENAI_API_KEY"),
# )
#
# # iterate over the test set and save the response for each prompt in an array
# for prompt in X_test_vignette_prompt:
#     response = client.responses.create(
#         model = "gpt-4o",
#         instructions = vignette_instruction,
#         input = prompt,
#     )
#     vignette_array_GPT.append(response.output_text)
#     print(response.output_text)
#
# # value counts for array
# counts_vignette_GPT = pd.Series(vignette_array_GPT).value_counts()
# print(counts_vignette_GPT)
#
# # convert YES to 1 and NO to 0
# vignette_array_GPT_val = [1 if response == "YES" else 0 for response in vignette_array_GPT]
# vignette_array_GPT_val
#
# # save the array to a csv file
# vignette_df_GPT = pd.DataFrame(vignette_array_GPT_val, columns = ["y_pred"])
# vignette_df_GPT.to_csv("../exp/preds_LLMs/y_pred_GPT4_vignette_prompt.csv", sep = ",", index = False)

NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO     245
YES      1
Name: count, dtype: int64


### 2.2 Gemini (Google)

In [48]:
# client = genai.Client(api_key = os.environ.get("GEMINI_API_KEY"))
#
# response = client.models.generate_content(
#     model = "gemini-2.0-flash",
#     contents = "Explain how AI works in a few words",
# )
#
# print(response.text)

In [46]:
client = genai.Client(api_key = os.environ.get("GEMINI_API_KEY"))

response = client.models.generate_content(
    model = "gemini-2.0-flash",
    config = types.GenerateContentConfig(
        system_instruction = simple_instruction),
    contents = simple_prompt
)

ValueError: Missing key inputs argument! To use the Google AI API, provide (`api_key`) arguments. To use the Google Cloud API, provide (`vertexai`, `project` & `location`) arguments.

### 2.3 Gemma (Google)

### 2.4 Llama (Meta)

### 2.5 Claude (Anthropic)

### 2.6 DeepSeek

In [37]:
# client = OpenAI(api_key = os.environ.get("DeepSeek_API_Key"), base_url = "https://api.deepseek.com")
#
# response = client.chat.completions.create(
#     model="deepseek-chat",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant"},
#         {"role": "user", "content": "Hello"},
#     ],
#     stream=False
# )
#
# print(response.choices[0].message.content)